In [1]:
import polars as pl
import glob
import os

from haystack import Pipeline
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter

from milvus_haystack import MilvusDocumentStore
from milvus_haystack.milvus_embedding_retriever import MilvusEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

In [2]:
df = pl.read_parquet('hf://datasets/AgentPublic/piaf/plain_text/train-00000-of-00001.parquet')

In [6]:
df.head()

id,title,context,question,answers
str,str,str,str,struct[2]
"""p140295443291664""","""Sport""","""Les dépenses des ménages repré…","""Combien de personnes travaille…","{[""100 000""],[472]}"
"""p140295443291520""","""Sport""","""Les dépenses des ménages repré…","""Combien d'employeurs""","{[""20 000""],[597]}"
"""p140295443291376""","""Sport""","""Les dépenses des ménages repré…","""Quel part du budget des ménage…","{[""50""],[46]}"
"""p140295443291088""","""Sport""","""Les dépenses des ménages repré…","""Quel montant en 2003""","{[""14,2 milliards""],[68]}"
"""p140295443290872""","""Sport""","""Les dépenses des ménages repré…","""Quel montant en 2019""","{[""12 milliards""],[102]}"


## Dataset Analysis

In [4]:
print(df.columns)

['id', 'title', 'context', 'question', 'answers']


In [5]:
df.describe()

statistic,id,title,context,question,answers
str,str,str,str,str,f64
"""count""","""3835""","""3835""","""3835""","""3835""",3835.0
"""null_count""","""0""","""0""","""0""","""0""",0.0
"""mean""",null,null,null,null,null
"""std""",null,null,null,null,null
"""min""","""p140295201616088""","""6 Heures de Shanghai 2017""","""2012 est sorti en 2012. Son th…","""A cause de qui Emanuele se voi…",null
"""25%""",null,null,null,null,null
"""50%""",null,null,null,null,null
"""75%""",null,null,null,null,null
"""max""","""p140295460357824""","""Événement Azolla""","""Étienne Báthory, roi de Pologn…","""à quelle ronde fénix est sorti…",null


In [31]:
# Indiquer le nombre de valeur nulle dans chaque colonne
print(df.null_count())

# Indiquer le nombre de lignes
print(df.height)

shape: (1, 5)
┌─────┬───────┬─────────┬──────────┬─────────┐
│ id  ┆ title ┆ context ┆ question ┆ answers │
│ --- ┆ ---   ┆ ---     ┆ ---      ┆ ---     │
│ u32 ┆ u32   ┆ u32     ┆ u32      ┆ u32     │
╞═════╪═══════╪═════════╪══════════╪═════════╡
│ 0   ┆ 0     ┆ 0       ┆ 0        ┆ 0       │
└─────┴───────┴─────────┴──────────┴─────────┘
3835


In [32]:
# keep 2500 rows
df = df.head(2500)

# Shuffle the dataframe
df_shuffled = df.sample(fraction=1.0, with_replacement=False, seed=42)

# Calculate split indices
n = df_shuffled.height
train_end = int(n * 0.7)
val_end = train_end + int(n * 0.15)

# Split the dataframe
train_df = df_shuffled[:train_end]
val_df = df_shuffled[train_end:val_end]
test_df = df_shuffled[val_end:]

In [33]:
from haystack import Document

# Create document lists for each dataset split
train_documents = [
    Document(
        content=row['context'],
        meta={'id': row['id'], 'title': row['title']}
    )
    for row in train_df.to_dicts()
]

val_documents = [
    Document(
        content=row['context'],
        meta={'id': row['id'], 'title': row['title']}
    )
    for row in val_df.to_dicts()
]

test_documents = [
    Document(
        content=row['context'],
        meta={'id': row['id'], 'title': row['title']}
    )
    for row in test_df.to_dicts()
]

# Use training documents for indexing
documents = test_documents

In [ ]:
embedding_model_list = [
    "intfloat/multilingual-e5-large-instruct",
    "Lajavaness/bilingual-embedding-large",
    "HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1",
]

In [9]:
for i, model in enumerate(embedding_model_list):
    print(f"Model {i+1}: {model}")
    embedder = SentenceTransformersDocumentEmbedder(model = model, trust_remote_code=True)
    embedder.warm_up()

    document_store = MilvusDocumentStore(
        connection_args={"uri": "./milvus.db"},
        drop_old=True,
        collection_name=f"piaf_{i+1}"
    )
    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("embedder", embedder)
    indexing_pipeline.add_component("writer", DocumentWriter(document_store))
    indexing_pipeline.connect("embedder", "writer")
    indexing_pipeline.run({"documents": documents})
    print(f"Indexed {len(documents)} documents with model {model}")

Model 1: intfloat/multilingual-e5-large-instruct


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Indexed 375 documents with model intfloat/multilingual-e5-large-instruct
Model 2: Lajavaness/bilingual-embedding-large


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Indexed 375 documents with model Lajavaness/bilingual-embedding-large
Model 3: HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

PipelineRuntimeError: The following component failed to run:
Component name: 'embedder'
Component type: 'SentenceTransformersDocumentEmbedder'
Error: MPS backend out of memory (MPS allocated: 35.95 GB, other allocations: 2.12 MB, max allowed: 36.27 GB). Tried to allocate 333.06 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [11]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.evaluators import DocumentMAPEvaluator, DocumentMRREvaluator, DocumentRecallEvaluator, DocumentNDCGEvaluator
from haystack import Document, Pipeline


for i, model in enumerate(embedding_model_list):
    grounds_truth = []
    retrieval_results_list = []
    print(f"Model {i+1}: {model}")
    embedder = SentenceTransformersTextEmbedder(model=model, trust_remote_code=True, progress_bar=False)
    embedder.warm_up()

    document_store = MilvusDocumentStore(
        connection_args={"uri": "./milvus.db"},
        collection_name=f"piaf_{i+1}"
    )

    retrieval_pipeline = Pipeline()
    retrieval_pipeline.add_component("embedder", embedder)
    retrieval_pipeline.add_component("retriever", MilvusEmbeddingRetriever(document_store=document_store, top_k=3))
    retrieval_pipeline.connect("embedder", "retriever")

    for row in test_df.to_dicts():
        retrieval_results = retrieval_pipeline.run({"embedder": {"text": row["question"]}})
        grounds_truth.append([Document(
            content=row["context"],
            meta={
                "id": row["id"],
                "title": row["title"],
            }
        )])
        retrieval_result = retrieval_results["retriever"]["documents"]
        retrieval_results_list.append(retrieval_result)
    evaluator = Pipeline()
    mrr_evaluator = DocumentMRREvaluator()
    map_evaluator = DocumentMAPEvaluator()
    recall = DocumentRecallEvaluator()
    ndcg = DocumentNDCGEvaluator()
    evaluator.add_component("mrr_evaluator", mrr_evaluator)
    evaluator.add_component("map_evaluator", map_evaluator)
    evaluator.add_component("recall_evaluator", recall)
    evaluator.add_component("ndcg_evaluator", ndcg)
    score = evaluator.run({
        "mrr_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth},
        "map_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth},
        "recall_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth},
        "ndcg_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth}
    }
    )
    print(f"Score for model {model}: ")
    print(f"MRR: {score['mrr_evaluator']['score']}")
    print(f"MAP: {score['map_evaluator']['score']}")
    print(f"Recall: {score['recall_evaluator']['score']}")
    print(f"NDCG: {score['ndcg_evaluator']['score']}")

Model 1: intfloat/multilingual-e5-large-instruct
Score for model intfloat/multilingual-e5-large-instruct: 
MRR: 0.6666666666666666
MAP: 0.6666666666666666
Recall: 0.6666666666666666
NDCG: 0.0
Model 2: Lajavaness/bilingual-embedding-large
Score for model Lajavaness/bilingual-embedding-large: 
MRR: 0.6693333333333333
MAP: 0.6693333333333333
Recall: 0.6693333333333333
NDCG: 0.0
Model 3: HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1
Score for model HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1: 
MRR: 0.648
MAP: 0.648
Recall: 0.648
NDCG: 0.0


In [13]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack import Pipeline

# 1. DocumentStore en mémoire avec BM25
document_store = InMemoryDocumentStore()
document_store.write_documents(documents)

retriever = InMemoryBM25Retriever(document_store=document_store)
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("retriever", retriever)


grounds_truth = []
retrieval_results_list = []

for row in test_df.to_dicts():
    retrieval_results = retrieval_pipeline.run({"retriever": {"query": row["question"]}})
    grounds_truth.append([Document(
        content=row["context"],
        meta={
            "id": row["id"],
            "title": row["title"],
        }
    )])
    retrieval_result = retrieval_results["retriever"]["documents"]
    retrieval_results_list.append(retrieval_result)

In [14]:
evaluator = Pipeline()
mrr_evaluator = DocumentMRREvaluator()
map_evaluator = DocumentMAPEvaluator()
recall = DocumentRecallEvaluator()
ndcg = DocumentNDCGEvaluator()
evaluator.add_component("mrr_evaluator", mrr_evaluator)
evaluator.add_component("map_evaluator", map_evaluator)
evaluator.add_component("recall_evaluator", recall)
evaluator.add_component("ndcg_evaluator", ndcg)
score = evaluator.run({
    "mrr_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth},
    "map_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth},
    "recall_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth},
    "ndcg_evaluator": {"retrieved_documents": retrieval_results_list, "ground_truth_documents": grounds_truth}
}
)
print(f"Score for model {model}: ")
print(f"MRR: {score['mrr_evaluator']['score']}")
print(f"MAP: {score['map_evaluator']['score']}")
print(f"Recall: {score['recall_evaluator']['score']}")
print(f"NDCG: {score['ndcg_evaluator']['score']}")

Score for model HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1: 
MRR: 0.6371111111111107
MAP: 0.6576162257495586
Recall: 0.7306666666666667
NDCG: 0.39933996689679263


In [8]:
output_path = "piaf_results"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from datasets import Dataset

# 1. Modèle de base
base_model = "intfloat/multilingual-e5-large-instruct"
model = SentenceTransformer(base_model)

# 2. Préparation des données
train_examples = [
    {"anchor": row["question"], "positive": row["context"]}
    for row in train_df.to_dicts()
]
train_dataset = Dataset.from_list(train_examples)

val_queries = {f"q{i}": row["question"] for i, row in enumerate(val_df.to_dicts())}
val_corpus  = {f"d{i}": row["context"]  for i, row in enumerate(val_df.to_dicts())}
val_rel     = {qid: {did: 1} for qid, did in zip(val_queries, val_corpus)}

evaluator = InformationRetrievalEvaluator(
    val_queries, val_corpus, val_rel, name="piaf-validation"
)

# 3. Loss
train_loss = MultipleNegativesRankingLoss(model)

# 4. Hyper-paramètres adaptés à MPS
training_args = SentenceTransformerTrainingArguments(
    output_dir=output_path,                # dossier de sortie
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    # Désactivation explicite de toute mixed precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,
)

# 5. Création du Trainer et lancement
trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    evaluator=evaluator,
    loss=train_loss
)
trainer.train()

# 6. Sauvegarde
model.save_pretrained(output_path)
print(f"✓ Modèle entraîné et sauvegardé dans : {output_path}")

## Data Augmentation - Query Generation

In [8]:
# Exploser les chunks du contexte et créer un dataset avec id, text, et title
chunks_data = []
for row in df.to_dicts():
    # Séparer les chunks par double saut de ligne
    chunks = row['context'].split('\n\n')
    for chunk in chunks:
        if chunk.strip():  # Ignorer les chunks vides
            chunks_data.append({
                'id': f"{row['id']}_{len(chunks_data)}",  # Créer un ID unique pour chaque chunk
                'text': chunk.strip(),
                'title': row['title']
            })

# Créer le dataframe Polars et supprimer les doublons
chunks_df = pl.DataFrame(chunks_data).unique()

chunks_df

id,text,title
str,str,str
"""p140295442538712_562""","""Les élections de 1990 portent …","""Pakistan"""
"""p140295202442552_3062""","""L'UAZ-469 et ses modifications…","""Ulyanovsky Avtomobilny Zavod"""
"""p140295442729432_245""","""Deux jours plus tard, malgré c…","""Guerre d'indépendance espagnol…"
"""p140295222229464_1048""","""La Première Guerre mondiale es…","""Histoire de l'Acadie"""
"""p140295442741072_265""","""Un autre concept utile est cel…","""Écologie comportementale"""
…,…,…
"""p140295442429416_453""","""Les années 1930 voient la mise…","""Moscou"""
"""p140295203784240_1754""","""Les présentations des films ho…","""Hongrie"""
"""p140295203385992_2409""","""Les directeurs du marketing on…","""Système d'information marketin…"


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="lmstudio-community/granite-3.1-8b-instruct-GGUF",base_url="http://localhost:1234/v1", api_key="not-needed"))
generator_embeddings = LangchainEmbeddingsWrapper(HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-multilingual-base",model_kwargs={"trust_remote_code" : True}))

README.md:   0%|          | 0.00/123k [00:00<?, ?B/s]

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
async def convert_prompt_to_french(extractor, llm):
    """Convertit les prompts d'un extracteur en français et les charge ou les adapte."""
    prompt_path = f".cacha/ragas/french/{extractor.property_name}/"

    try: 
        new_prompt = extractor.load_prompts(prompt_path, language="french")
        print(f"{extractor.__class__.__name__} prompts loaded.")
    except Exception as e:
        new_prompt = await extractor.adapt_prompts(language="french", llm=llm)
        extractor.set_prompts(**new_prompt)
        
        if not os.path.exists(prompt_path):
            os.makedirs(prompt_path)
        extractor.save_prompts(prompt_path)
        print(f"{extractor.__class__.__name__} prompts saved.")
    else:
        extractor.set_prompts(**new_prompt)

In [11]:
from ragas.testset.graph import NodeType
from ragas.testset.transforms.extractors import EmbeddingExtractor,HeadlinesExtractor,SummaryExtractor
from ragas.testset.transforms.extractors.llm_based import NERExtractor, ThemesExtractor
from ragas.testset.transforms.filters import CustomNodeFilter
from ragas.testset.transforms.relationship_builders import CosineSimilarityBuilder,OverlapScoreBuilder
from ragas.testset.transforms.splitters import HeadlineSplitter
from ragas.testset.transforms.engine import Parallel


headline_extractor = HeadlinesExtractor(llm=generator_llm)
await convert_prompt_to_french(headline_extractor, generator_llm)
theme_extractor = ThemesExtractor(llm=generator_llm)
await convert_prompt_to_french(theme_extractor, generator_llm)
ner_extractor = NERExtractor(
        llm=generator_llm, filter_nodes=lambda node: node.type == NodeType.CHUNK
    )
await convert_prompt_to_french(ner_extractor, generator_llm)
summary_extractor = SummaryExtractor(
        llm=generator_llm
    )
await convert_prompt_to_french(summary_extractor, generator_llm)
splitter = HeadlineSplitter(min_tokens=500)
content_emb_extractor = EmbeddingExtractor(
        embedding_model=generator_embeddings,
        property_name="content_embedding",
        embed_property_name="page_content",
    )
cosine_sim_builder = CosineSimilarityBuilder(
        property_name="content_embedding",
        new_property_name="content_similarity",
        threshold=0.7,
    )
ner_overlap_sim = OverlapScoreBuilder(
        threshold=0.01, filter_nodes=lambda node: node.type == NodeType.CHUNK
    )
node_filter = CustomNodeFilter(llm=generator_llm, filter_nodes=lambda node: node.type == NodeType.CHUNK)


transforms = [
        headline_extractor,
        #splitter,
        #summary_extractor,
        node_filter,
        Parallel(content_emb_extractor, theme_extractor, ner_extractor),
        Parallel(cosine_sim_builder, ner_overlap_sim),
    ] 

APIConnectionError: Connection error.

In [ ]:
from ragas.testset.persona import Persona
default_persona = Persona(
    name="Utilisateur lambda",
    role_description="Parle français et demande des simples renseignements dans le domaine de la fonction public."
)

personas = [
    default_persona
]

In [12]:
from ragas.testset.synthesizers.multi_hop import MultiHopAbstractQuerySynthesizer,MultiHopSpecificQuerySynthesizer
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer

query_distribution =  [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
    ]

In [13]:
from ragas.testset.graph import Node
from ragas.testset import TestsetGenerator
from langchain_community.callbacks.manager import get_openai_callback
import os
import datetime
import time 
import json
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType
from ragas.testset.transforms import apply_transforms
import random

kg = KnowledgeGraph()


# Add the first X documents to the KG
for i, row in enumerate(chunks_df.to_dicts()):
    kg.nodes.append(
        Node(
            type=NodeType.CHUNK,
            properties={"page_content": row['text'], "id": row['id'], "title": row['title']},
        )
    )

In [15]:
kg

KnowledgeGraph(nodes: 3835, relationships: 0)

In [ ]:
kg = kg.load("kg.json")

In [ ]:
with get_openai_callback() as cb:
    generator = TestsetGenerator(llm=generator_llm, knowledge_graph=kg,persona_list=personas, embedding_model=generator_embeddings)
    testset = generator.generate(testset_size=130,num_personas=1,query_distribution=query_distribution)
    df_results = testset.to_pandas()
    print(f"Total tokens used: {cb.total_tokens}")
    print(f"Total cost (in USD): {cb.total_cost}")

## Indexing data

In [34]:
final_model_name = "intfloat/multilingual-e5-large-instruct"

In [ ]:
embedder = SentenceTransformersDocumentEmbedder(model = final_model_name, trust_remote_code=True)
embedder.warm_up()

document_store = MilvusDocumentStore(
    connection_args={"uri": "./milvus.db"},
    drop_old=True,
    collection_name="piaf_final_model"
)
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("embedder", embedder)
indexing_pipeline.add_component("writer", DocumentWriter(document_store))
indexing_pipeline.connect("embedder", "writer")
indexing_pipeline.run({"documents": train_documents})
indexing_pipeline.run({"documents": val_documents})
indexing_pipeline.run({"documents": test_documents})
print(f"Indexed {len(documents)} documents with model {model}")

Batches:   0%|          | 0/55 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Indexed 375 documents with model SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
